In [1]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install pyti
# !pip install akshare

In [2]:
from data_provider.data_creat import *
import akshare as ak
from datetime import datetime
import random

In [3]:
class Args:
    '''股票数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20151201'  # 20151201   20221021  下载数据的开始日期,1就是公司上市时间
    end_date = '20231230'  # '20231220' 下载数据的结束日期,如果1则到最后一天,如果-1是昨天.
    root_path = './dataset/Stock/'
    
    # 添加预测目标Y
    label_n = 0 #  data['Tom_Chg'] 预测未来连续多少天的涨幅，0就是当天的涨幅。
    # 训练数据需要将end=lable_n，删除掉计算不出来的。预测数据不需要。
    end = 0 # 删除最后部分需要预测天数label_n的数据，算出来是0.
    zhangfu = 5  # data['OT']如果是非0，那么替换Y。涨幅大于等于5个点为1，小于5个点的为0
    
    # 是否合并全部股票数据
    all = True # 是否合并全部股票数据
    data_addzero = 21 # 当all = True时，用于训练的数据集分割，前面补0的长度 应该是seq_len + pred_len 长度
    
    # 数据修剪
    start = 103 # 删除前24行（start=25），因为macd算不出来
    # final_data_feat =  ['index', 'Volume','Tom_Chg'] # 删除不需要列的标签
    final_data_feat =  ['index', 'Volume','Tom_Chg','Open','Low','High'] # 删除不需要列的标签
    down_scaler = False # 是否对下载数据进行归一化
    
    # 此项用于预测数据，训练数据不需要
    add_zero_days = 0 # 未来数据填充0的天数
    pred_path = './dataset/Stock/pred/'

# 创建参数对象
args = Args()

# 创建股票数据列表
stock_down = ak.stock_cy_a_spot_em() # 创业板实时数据
stock_list = stock_down[~stock_down['名称'].str.contains("退|ST") & (stock_down['流通市值'] <= 1e11) & (stock_down['总市值'] >= 45e8)] # 去除退市和ST股票
file_name_cy = 'Stock_list_cy.csv'# 保存数据，编码格式为utf-8
stock_list.to_csv(args.root_path + file_name_cy,index=False,encoding='utf-8-sig')

# 读取股票列表
stock_list = pd.read_csv(args.root_path + file_name_cy) # 读取股票列表
# 将股票代码的数字转换为字符串列表
stock_list = [str(code) for code in stock_list['代码'].tolist()]
# 对列表随机排序
random.shuffle(stock_list)

# stock_list = ['000158','300798'] # 自定义股票列表

In [4]:
from sklearn.preprocessing import StandardScaler

all_data_raw = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储原始数据
all_data_scaled = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储标准化后的数据
processed_count = 0  # 初始化计数器
total_count = len(stock_list)  # 获取总股票数量
scaler = StandardScaler()

for i in stock_list[:]:
    NUM = i
    try:
        # 下载原始数据
        raw_data = download_data(NUM, args)
        # 更新已处理股票数量计数器
        processed_count += 1
        # 检查数据长度，如果小于300则跳过此次循环
        if raw_data.shape[0] < 500:
            print(f"股票代码 {NUM} 的数据长度小于300,跳过此次循环。")
            continue
        
        # 拼接数据，添加各种参数
        ad_data = add_data(raw_data, args)
        # 添加预测标签
        ad_data = add_label(ad_data, args)
        # 删除无效数据
        final_data_raw = sub_data(ad_data.copy(), args)
        final_data_scaled = final_data_raw.copy()
        
        # 未处理数据的前20行替换为0
        if args.data_addzero and not all_data_raw.empty:
            final_data_raw = add_zeros_to_data(final_data_raw, num_rows=args.data_addzero)

        # 标准化处理
        if args.down_scaler:
            non_time_columns = final_data_scaled.columns[1:-1]  # 假设时间列是第一列
            final_data_scaled[non_time_columns] = scaler.fit_transform(final_data_scaled[non_time_columns])
            # 标准化处理的数据的前20行替换为0
            if args.data_addzero and not all_data_scaled.empty:
                final_data_scaled = add_zeros_to_data(final_data_scaled, num_rows=args.data_addzero)
        
        # 如果 all 为 False, 则为每个股票单独保存数据
        if not args.all:
            file_name_raw_individual = f"raw_stock_cy_{NUM}.csv"
            final_data_raw.to_csv(args.root_path + file_name_raw_individual, index=False)

            if args.down_scaler:
                file_name_scaled_individual = f"scaled_stock_cy_{NUM}.csv"
                final_data_scaled.to_csv(args.root_path + file_name_scaled_individual, index=False)
                print(f"股票代码 {NUM} 的数据保存到 {file_name_raw_individual} ，形状: {final_data_raw.shape}")
        else:
            # 拼接未处理的数据
            all_data_raw = pd.concat([all_data_raw, final_data_raw], ignore_index=True)
            if args.down_scaler:
                # 拼接经过标准化的数据
                all_data_scaled = pd.concat([all_data_scaled, final_data_scaled], ignore_index=True)
            print(f"当前all_data的形状: {all_data_raw.shape}")

    except Exception as e:
        print(f"处理股票代码 {NUM} 时出现错误: {e}")
        continue

    # 计算并打印处理进度
    progress = processed_count / total_count
    print(f"处理进度: {processed_count}/{total_count} ({progress:.2%})")

# 如果 all 为 True, 则保存合并后的数据
if args.all:
    time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    file_name_raw = "stock_cy_all_raw.csv"
    all_data_raw.to_csv(args.root_path + time + file_name_raw, index=False)
    if args.down_scaler:
        file_name_scaled = "stock_cy_all_scaled.csv"
        all_data_scaled.to_csv(args.root_path + time + file_name_scaled, index=False)
    print("合并数据保存完毕。")
else:
    print("单独数据保存完毕。")

获取数据时间为： 20151201 - 20231230
原始数据形状： (512, 9)
添加数据以后形状： (512, 23)
添加label以后数据形状: (512, 25)
删除指定行、列后数据形状:  (409, 20)
当前all_data的形状: (409, 20)
处理进度: 1/591 (0.17%)
获取数据时间为： 20151201 - 20231230
原始数据形状： (522, 9)
添加数据以后形状： (522, 23)
添加label以后数据形状: (522, 25)
删除指定行、列后数据形状:  (419, 20)
当前all_data的形状: (828, 20)
处理进度: 2/591 (0.34%)
获取数据时间为： 20151201 - 20231230
原始数据形状： (1833, 9)
添加数据以后形状： (1833, 23)
添加label以后数据形状: (1833, 25)
删除指定行、列后数据形状:  (1730, 20)
当前all_data的形状: (2558, 20)
处理进度: 3/591 (0.51%)
获取数据时间为： 20151201 - 20231230
原始数据形状： (1623, 9)
添加数据以后形状： (1623, 23)
添加label以后数据形状: (1623, 25)
删除指定行、列后数据形状:  (1520, 20)
当前all_data的形状: (4078, 20)
处理进度: 4/591 (0.68%)
获取数据时间为： 20151201 - 20231230
原始数据形状： (610, 9)
添加数据以后形状： (610, 23)
添加label以后数据形状: (610, 25)
删除指定行、列后数据形状:  (507, 20)
当前all_data的形状: (4585, 20)
处理进度: 5/591 (0.85%)
获取数据时间为： 20151201 - 20231230
原始数据形状： (1718, 9)
添加数据以后形状： (1718, 23)
添加label以后数据形状: (1718, 25)
删除指定行、列后数据形状:  (1615, 20)
当前all_data的形状: (6200, 20)
处理进度: 6/591 (1.02%)
获取数据时间为： 20151201 